#Ahihiiii

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
import shutil
from ninhnq1911_mltools.file_util import get_resource
from huggingface_hub import hf_hub_download


hf_dataset_identifier = "sayakpaul/ucf101-subset"
filename = "UCF101_subset.tar.gz"
file_path = hf_hub_download(
    repo_id=hf_dataset_identifier, filename=filename, repo_type="dataset"
)
print(file_path)
shutil.copy(file_path, get_resource("UCF101_subset.tar.gz"))

ModuleNotFoundError: No module named 'file_util'

In [26]:
%pip install wget

  Using cached wget-3.2.zip (10 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement gzip (from versions: none)
ERROR: No matching distribution found for gzip


In [34]:
import tarfile

tarfile.open(get_resource("UCF101_subset.tar.gz")).extractall(get_resource("UCF101_subset"), filter='data')

DATA/VIDEOMAE/UCF101_subset.tar.gz
